In [1]:
## Instalaciones

%pip install torch
%pip install open3d


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
## Dependencias

import torch
import os
import open3d as o3d
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader
from model import PointnetClassifier, PointNetLoss
from modelnet10 import ModelNetClass, ModelNet, DatasetType
from utils.csv import save_loss_dict

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {DEVICE}.")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Using cpu.


/home/antonioskorin/.local/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
# parámetros de ejecución
checkpoint_freq = 5        # cada cuantos epoch guardar el modelo
ROOT_DIR = os.getcwd()
CHECKPOINT_DIR = os.path.join(ROOT_DIR, "checkpoint")

# parámetros del dataset
classes = [label for label in ModelNetClass]
batch_size = 32
dim = 3
num_points = 1024
num_classes = len(classes)

# hiperparámetros
num_global_feats = 1024     # número de features globales calculadas
epochs = 10
learning_rate = 0.001
reg_weight = 0.001
gamma = 2                   # Recomendado por el paper de focal loss


train_data = ModelNet(classes, DatasetType.TRAIN)
validation_data = ModelNet(classes, DatasetType.VALIDATION)
test_data = ModelNet(classes, DatasetType.TEST)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

classifier = PointnetClassifier(dim, num_points, num_global_feats, num_classes).to(DEVICE)

optimizer = optim.Adam(classifier.parameters(), lr=learning_rate)

if DEVICE == "cuda":
    scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0001, max_lr=0.01, step_size_up=2000, cycle_momentum=False)
else:
    scheduler = None
    
# TODO: Más adelante usar alpha para clases imbalanceadas
criterion = PointNetLoss(alpha=None, gamma=gamma, reg_weight=reg_weight, size_average=True).to(DEVICE)

In [4]:
loss_dict = {
    "train": {
        "loss": list(),
        "acc": list()
    },
    "valid": {
        "loss": list(),
        "acc": list()
    }
}

for epoch in range(1, epochs + 1):
    print(f"Epoch {epoch}:")
    '''
    Entrenamiento
    '''
    classifier = classifier.train()

    train_batch_losses = list()
    train_batch_accs = list()
    
    for pcds, labels in train_loader:
        pcds = pcds.to(DEVICE)
        labels = labels.squeeze().to(DEVICE)

        # Gradientes en cero
        optimizer.zero_grad()
        
        # Hacer predicciones, calcular pérdida
        out, _, A = classifier(pcds)
        loss = criterion(out, labels, A, is_train=True)

        # Calcular gradiente y optimizar
        loss.backward()
        optimizer.step()
        
        # Uso de scheduler para el LR, en caso de estar habilitado
        if DEVICE == "cuda":
            scheduler.step()

        with torch.no_grad():
            # Calculamos las elecciones
            pred_choice = torch.softmax(out, dim=1).argmax(dim=1)
            
            # Elecciones correctas
            correct = pred_choice.eq(labels.data).cpu().sum()
            accuracy = correct.item() / float(pcds.size(0))
                
    
        # Registramos métricas para el batch
        train_batch_losses.append(loss.item())
        train_batch_accs.append(accuracy)
        

    '''
    Validación
    '''
    with torch.no_grad():
        classifier = classifier.eval()

        valid_batch_losses = list()
        valid_batch_accs = list()
        
        for pcds, labels in validation_loader:
            pcds = pcds.to(DEVICE)
            labels = labels.squeeze().to(DEVICE)
            
            # Hacer predicciones
            out, _, A = classifier(pcds)
            loss = criterion(out, labels, A, is_train=False)

            # Calculamos las elecciones
            pred_choice = torch.softmax(out, dim=1).argmax(dim=1)
            
            # Elecciones correctas
            correct = pred_choice.eq(labels.data).cpu().sum()
            accuracy = correct.item() / float(pcds.size(0))

            # Registramos métricas para el batch
            valid_batch_losses.append(loss.item())
            valid_batch_accs.append(accuracy)

    # Loss y pérdida training
    epoch_train_loss = np.mean(train_batch_losses)
    epoch_train_acc = np.mean(train_batch_accs)
    print("  Train Loss:\t\t", epoch_train_loss)
    print("  Train Acc:\t\t", epoch_train_acc)
    loss_dict["train"]["loss"].append(epoch_train_loss)
    loss_dict["train"]["acc"].append(epoch_train_acc)
    # Loss y pérdida validation
    epoch_valid_loss = np.mean(valid_batch_losses)
    epoch_valid_acc = np.mean(valid_batch_accs)
    print("  Validation Loss:\t", epoch_valid_loss)
    print("  Validation Acc:\t", epoch_valid_acc)
    loss_dict["valid"]["loss"].append(epoch_valid_loss)
    loss_dict["valid"]["acc"].append(epoch_valid_acc)

    if epoch % checkpoint_freq == 0:
        path = os.path.join(CHECKPOINT_DIR, f"model_epoch_{str(epoch).zfill(4)}.pth")
        torch.save(classifier.state_dict(), path)
    

save_loss_dict(loss_dict)

Epoch 1:
  Train Loss:		 1.8653536729812623
  Train Acc:		 0.3179891304347826
  Validation Loss:	 52.22469165563584
  Validation Acc:	 0.25
Epoch 2:
  Train Loss:		 1.3587088408470154
  Train Acc:		 0.4426195652173913
  Validation Loss:	 14461.367341275216
  Validation Acc:	 0.1475
Epoch 3:
  Train Loss:		 1.2312325463294982
  Train Acc:		 0.5182173913043479
  Validation Loss:	 2.926939141750336
  Validation Acc:	 0.2525
Epoch 4:
  Train Loss:		 1.0695222754478455
  Train Acc:		 0.5576195652173912
  Validation Loss:	 3.2634958171844484
  Validation Acc:	 0.1525
Epoch 5:
  Train Loss:		 0.9173334140777588
  Train Acc:		 0.5913695652173913
  Validation Loss:	 204.27920606851578
  Validation Acc:	 0.055
Epoch 6:
  Train Loss:		 1.1446454961299897
  Train Acc:		 0.5983804347826087
  Validation Loss:	 23.78810644119978
  Validation Acc:	 0.1175
Epoch 7:
  Train Loss:		 0.8683892385959625
  Train Acc:		 0.6176739130434783
  Validation Loss:	 20.29452622413635
  Validation Acc:	 0.11
Epoch 8:

# Dataset Testeo

In [6]:
with torch.no_grad():
    classifier = classifier.eval()
    correct = 0
    
    for pcds, labels in test_loader:
        pcds = pcds.to(DEVICE)
        labels = labels.squeeze().to(DEVICE)
        
        # Hacer predicciones
        out, _, _ = classifier(pcds)
    
        # Calculamos las elecciones
        pred_choice = torch.softmax(out, dim=1).argmax(dim=1)
        
        # Elecciones correctas, acumuladas
        correct += pred_choice.eq(labels.data).cpu().sum()

    test_acc = correct / float(len(test_data))
    print("Test Acc:\t", test_acc)

Test Acc:	 tensor(0.0892)
